1. load_boston 데이터셋에서 data, target부분 나눠서 데이터프레임 만들기
2. train set, test set 분리
3. Pipeline을 이용해 다항회귀, 선형회귀 만들고 fit, predict
4. 평가된 모델의 mse,rmse, r2 score 확인

In [5]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

boston = load_boston()
df = pd.DataFrame(boston.data, columns = boston.feature_names)

df['Price'] = boston.target
X_data = df.drop('Price', axis=1)
y_target = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X_data, y_target,
                                                   test_size=0.3,
                                                   random_state=11)

model = Pipeline([('poly' , PolynomialFeatures(degree=2, include_bias=False)),
                  ('linear', LinearRegression())])

model.fit(X_train, y_train)
pred = model.predict(X_test)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)

print(f'MSE : {mse : .4f}, RMSE : {rmse : .4f}')
print(f'R2 score : {r2_score(y_test, pred) : .4f}')

MSE :  15.3887, RMSE :  3.9228
R2 score :  0.8316


### get_scaled_data(method=None, p_degree, df)
- input으로 들어온 scaling 방법을 적용해 fit_transform
- input으로 p_degree가 들어오면 polynomialFeatures 변환

In [9]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures

def get_scaled_data(method=None, p_degree = None, df=None):
    if method == 'Standard':
        scaled_data = StandardScaler().fit_transform(df)
    elif method =='MinMax':
        scaled_data = MinMaxScaler().fit_transform(df)
    elif method =='Log':
        scaled_data = np.log1p(df)
    else:
        scaled_data = df
        
    if p_degree!=None:
        scaled_data = PolynomialFeatures(degree=p_degree, include_bias=False).fit_transform(df)
        
    return scaled_data

### get_linear_reg_eval(model_name, params, X_data, y_target, verbose=True)
- 배열형태로 들어오는 params 각각 원소마다 input으로 받은 model_name에 맞는 규제방법에 alpha값을 적용
- cross_val_score을 통해 neg_mse_score, avg_rmse를 구함
- 결과를 데이터 프레임 형태로 보기위해 fit을 한후, model.coef_(회귀계수)를 데이터프레임에 data로, 인덱스에는 해당하는 feature명을 넣어줌

In [10]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score

def get_linear_reg_eval(model_name, params=None, X_data_n=None, y_target_n=None, verbose=True):
    eval_df = pd.DataFrame()
    if verbose:
        print('###### ', model_name, '######')
    for param in params:
        if model_name=='Ridge': model=Ridge(alpha=param)
        elif model_name=='Lasso' : model=Lasso(alpha=param)
        elif model_name=='ElasticNet' : model=ElasticNet(alpha=param, l1_ratio=0.7)
        neg_mse_scores = cross_val_score(model, X_data_n, y_target_n, 
                                        scoring="neg_mean_squared_error", cv=5)
        avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
        print(f'alpha : {param}일 때 5 폴드 세트의 평균 RMSE : {avg_rmse : .3f}')
        
        model.fit(X_data, y_target)
        evals = pd.Series(data=model.coef_, index=X_data.columns)
        colname = 'alpha: ' + str(param)
        eval_df[colname] = evals
    return eval_df

### 평가
- scale_methods에 원하는 scaling과 polynomial 변환 셋을 설정
- X_data_scaled에 scale_methods 각각 원소를 scaling함
- scale된 데이터를 Ridge규제로 각 alpha값들에 대하여 평가 확인

In [11]:
alphas = [0.1, 1, 10, 100]
scale_methods = [(None, None), ('Standard', None), ('Standard', 2),
               ('MinMax', None), ('MinMax' ,2), ('Log',None)]
for scale_method in scale_methods:
    X_data_scaled = get_scaled_data(method=scale_method[0], p_degree=scale_method[1],
                                   df=X_data)
    print('\n## 변환 유형 : {0}, Polynomial Degree : {1}'.format(scale_method[0], scale_method[1]))
    get_linear_reg_eval('Ridge', params=alphas, X_data_n=X_data_scaled, y_target_n=y_target, verbose=False)


## 변환 유형 : None, Polynomial Degree : None
alpha : 0.1일 때 5 폴드 세트의 평균 RMSE :  5.788
alpha : 1일 때 5 폴드 세트의 평균 RMSE :  5.653
alpha : 10일 때 5 폴드 세트의 평균 RMSE :  5.518
alpha : 100일 때 5 폴드 세트의 평균 RMSE :  5.330

## 변환 유형 : Standard, Polynomial Degree : None
alpha : 0.1일 때 5 폴드 세트의 평균 RMSE :  5.826
alpha : 1일 때 5 폴드 세트의 평균 RMSE :  5.803
alpha : 10일 때 5 폴드 세트의 평균 RMSE :  5.637
alpha : 100일 때 5 폴드 세트의 평균 RMSE :  5.421

## 변환 유형 : Standard, Polynomial Degree : 2
alpha : 0.1일 때 5 폴드 세트의 평균 RMSE :  9.141
alpha : 1일 때 5 폴드 세트의 평균 RMSE :  8.938
alpha : 10일 때 5 폴드 세트의 평균 RMSE :  10.556
alpha : 100일 때 5 폴드 세트의 평균 RMSE :  10.566

## 변환 유형 : MinMax, Polynomial Degree : None
alpha : 0.1일 때 5 폴드 세트의 평균 RMSE :  5.764
alpha : 1일 때 5 폴드 세트의 평균 RMSE :  5.465
alpha : 10일 때 5 폴드 세트의 평균 RMSE :  5.754
alpha : 100일 때 5 폴드 세트의 평균 RMSE :  7.635

## 변환 유형 : MinMax, Polynomial Degree : 2
alpha : 0.1일 때 5 폴드 세트의 평균 RMSE :  9.141
alpha : 1일 때 5 폴드 세트의 평균 RMSE :  8.938
alpha : 10일 때 5 폴드 세트의 평균 RMSE :  10.556
alpha : 100일